In [ ]:
!pip install -q geemap
!pip install -U geemap


import ee
import geemap
import matplotlib.pyplot as plt

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="ee-azchtateen")

In [ ]:
# Proyeksi datar (planar) Web Mercator untuk membuat buffer lingkaran
geom = ee.Geometry.Point([112.9512, -7.9418]).buffer(10000)

# Fungsi masking awan dari citra Sentinel-2
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
           qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Koleksi citra sebelum kejadian (median composite)
before = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2021-01-01', '2021-11-03')
    .filterBounds(geom)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
    .map(maskS2clouds)
    .median()
    .clip(geom)
)

# Koleksi citra setelah kejadian (median composite)
after = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-09-06', '2023-11-03')
    .filterBounds(geom)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
    .map(maskS2clouds)
    .median()
    .clip(geom)
)

# Parameter visualisasi false-color SWIR
vis_params = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B12', 'B11', 'B4']
}

# Buat peta dan tampilkan layer
Map = geemap.Map(center=[-7.949, 112.9496], zoom=12)
Map.addLayer(geom, {'color': 'red'}, 'Bukit Teletubbies Buffer')
Map.addLayer(before, vis_params, 'Before')
Map.addLayer(after, vis_params, 'After')
Map

### Calculate NBR

In [ ]:
# Fungsi NBR
def compute_nbr(image):
    return image.expression(
        "(NIR - SWIR2) / (NIR + SWIR2)",
        {
            'NIR': image.select("B8"),
            'SWIR2': image.select("B12")
        }
    )

# Hitung indeks kebakaran (NBR)
before_nbr = compute_nbr(before)
after_nbr = compute_nbr(after)

# Tampilkan hasil
Map.addLayer(before_nbr, {'min': -1, 'max': 1}, "Burnt Indices Before", False)
Map.addLayer(after_nbr, {'min': -1, 'max': 1}, "Burnt Indices After", False)


# Tampilkan peta
Map


### Compute and Classify dNBR

In [ ]:
# Hitung dNBR
dnbr_unscaled = before_nbr.subtract(after_nbr)
dnbr = dnbr_unscaled.multiply(1000)

Map.addLayer(dnbr, {'min': -1000, 'max': 1000}, 'dNBR', False)
Map

In [ ]:
# Klasifikasi dNBR berdasarkan threshold
thresholds = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000])
classified = dnbr.lt(thresholds).reduce('sum').toInt()

# Definisikan palet warna sesuai dNBR klasifikasi
palette = [
    '#ffffff',
    '#a41fd6',  # 0: High Severity
    '#ff641b',  # 1: Moderate-high Severity
    '#ffaf38',  # 2: Moderate-low Severity
    '#fff70b',  # 3: Low Severity
    '#0ae042',  # 4: Unburned
    '#acbe4d',  # 5: Enhanced Regrowth, Low
    '#7a8737'   # 6: Enhanced Regrowth, High
]

pixstats = classified.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=geom,
    scale=10
)

total_pixels = ee.Number(pixstats.get('sum'))
names = ['NA', 'High Severity', 'Moderate-high Severity',
         'Moderate-low Severity', 'Low Severity', 'Unburned',
         'Enhanced Regrowth, Low', 'Enhanced Regrowth, High']

results = []
for i in range(8):
    mask = classified.eq(i)
    count = mask.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geom,
        scale=10
    ).get('sum')
    count = ee.Number(count)
    hectares = count.multiply(100).divide(10000)
    percentage = count.divide(total_pixels).multiply(10000).round().divide(100)
    results.append(ee.Dictionary({
        'Class': names[i],
        'Pixels': count,
        'Hectares': hectares,
        'Percentage': percentage
    }))

print('Burned Area by Severity Class:')
for r in results:
    print(r.getInfo())

# Visualisasikan hasil klasifikasi
Map.addLayer(classified, {'min': 0, 'max': 7, 'palette': palette}, 'dNBR Classified')
Map

In [ ]:
# @title Histogram and Chart
from geemap import chart

# Gabungkan dua citra NBR
stacked_nbr = before_nbr.addBands(after_nbr)

# Buat histogram gabungan
fig = chart.image_histogram(
    stacked_nbr,
    region=geom,
    scale=30,
    max_buckets=200,
    min_bucket_width=0.01,
    max_raw=1000,
    max_pixels=int(1e6),
    title="Histogram of NBR Before and After Fire",
    labels=["NBR Before", "NBR After"],
    colors=["#1d6b99", "#cf513e"],
)

fig  # Tampilkan plot


In [ ]:
# Definisikan label dan warna untuk legenda (tanpa NA)
legend_dict = {
    "Enhanced Regrowth, High": "#7a8737",
    "Enhanced Regrowth, Low": "#acbe4d",
    "Unburned": "#0ae042",
    "Low Severity": "#fff70b",
    "Moderate-low Severity": "#ffaf38",
    "Moderate-high Severity": "#ff641b",
    "High Severity": "#a41fd6"
}


# Tambahkan legenda ke peta
Map.add_legend(
    title="Burn Severity (dNBR)",
    legend_dict=legend_dict,
    position="bottomright"
)

# Tampilkan peta
Map

In [ ]:
# @title Export dNBR
task = ee.batch.Export.image.toDrive(
    image=dnbr,
    description='dNBR2019',
    scale=10,
    region=geom,
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True}
)
task.start()


In [ ]:
Map